In [1]:
# Part 1: Setup and get a dataframe
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
# Initialize Pyspark
spark = SparkSession.builder.appName('Practice').getOrCreate()

spark

23/05/28 11:34:05 WARN Utils: Your hostname, penguin resolves to a loopback address: 127.0.1.1; using 100.115.92.211 instead (on interface eth0)
23/05/28 11:34:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/28 11:34:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Create a dataframe from a csv. and print data if wanted.
# Option 1
df_pyspark = spark \
                .read \
                .option('header', 'true') \
                .csv('test.csv', inferSchema=True)
# Option 2
df_pyspark = spark.read.csv('test.csv', header=True, inferSchema=True)

23/05/28 11:43:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 143935 ms exceeds timeout 120000 ms
23/05/28 11:43:29 WARN SparkContext: Killing executors is not supported by current scheduler.
23/05/28 11:43:29 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B

In [12]:
# Part2
#f_pyspark.select('name').show()
#df_pyspark.select(['name','age']).show()
#df_pyspark.describe().show()
#df_pyspark.dtypes

# Adding column in df
df_pyspark = df_pyspark.withColumn('experience', df_pyspark['age'] - 10)

# Drop it
#df_pyspark = df_pyspark.drop('experience')

# Rename
#df_pyspark.withColumnRenamed('experience', 'experience again').show()NNb 
df_pyspark = df_pyspark.drop('experience')
df_pyspark.show()

+-----+---+---+------+
| name|age|exp|salary|
+-----+---+---+------+
| sean| 28|  1|  null|
| jake| 11|  3| 20000|
|henry| 40| 20|100000|
+-----+---+---+------+



# Dealing with Null Values and Simple Imputer

In [17]:
# how="any" Drops rows with any na.
# thresh=2 at least 2 non null values should be present, then it drops any.
df_pyspark.na.drop("any", thresh=2)#.show()

# subset will drop any row with a NA  inside that column
df_pyspark.na.drop("any", subset=["salary"])#.show()


DataFrame[name: string, age: int, exp: int, salary: int]

In [25]:
# Filling values with the first parameter.
# you an subset with the second for any column
df_pyspark.na.fill(42, ["salary"]).show()

# More importantly, we might want to fill it with a mean using imputer:
from pyspark.ml.feature import Imputer

# NOTE: You can do multiple..
imputer = Imputer(
    inputCols=["salary", "age"],
    outputCols=[f"{x}_imputed" for x in ["salary", "age"]]
).setStrategy("mean")

imputer.fit(df_pyspark).transform(df_pyspark).show()

+-----+---+---+------+
| name|age|exp|salary|
+-----+---+---+------+
| sean| 28|  1|    42|
| jake| 11|  3| 20000|
|henry| 40| 20|100000|
+-----+---+---+------+

+-----+---+---+------+--------------+-----------+
| name|age|exp|salary|salary_imputed|age_imputed|
+-----+---+---+------+--------------+-----------+
| sean| 28|  1|  null|         60000|         28|
| jake| 11|  3| 20000|         20000|         11|
|henry| 40| 20|100000|        100000|         40|
+-----+---+---+------+--------------+-----------+



# Filter Operations

In [11]:
# Single conditions
#df_pyspark.filter("age<=30").select(["name", "salary"]).show() # Etc...
df_pyspark.filter(df_pyspark["age"]<=30).select(["name", "salary"]).show() # Etc...

# Multiple conditions
# NOTE: Make sure you add the proper ()'s
df_pyspark.filter((df_pyspark["age"]<=30) & (df_pyspark["age"] > 20)).select(["name", "salary"]).show() # Etc...


+----+------+
|name|salary|
+----+------+
|sean|  null|
|jake| 20000|
+----+------+

+----+------+
|name|salary|
+----+------+
|sean|  null|
+----+------+



23/05/28 11:46:55 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:593)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:592)
	at org.apache.spar